# Overview

In this notebook we are going to create a SparkContext object. As we will see, this object is our communication channel with the Apache Spark cluster. It allows us to load data and execute code on the Spark cluster.

As discussed in the [README](README.md) we will see that creating the SparkContext that is configured to use kubernetes will automagically spin up a set of spark workers which run on the kubernetes cluster. The driver will run locally in "client mode".

Spark provides a Dashboard for monitoring the work being executed on the cluster. When we create the sparkContext from our jupyter notebook, a service will be spun up to listen on port 4040 (on the machine hosting the nupyter notebook. In my case, the dashboard was available at the following URL http://15.1.1.23:4040. 

When the cluster is first created, we can expand the event timeline to see when the driver and executors are added to the cluster.

<center><img src="images/spark_dashboard_event_timeline.png" width="600px"/></center>

Once jobs have been submitted we can see the jobs that are/have run. It resembles the following:

<center><img src="images/Apache%20Spark%20Dashboard.png" width="600px"/></center>

It assumes you have already read the following notebooks:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Running Apache Spark On Kubernetes](Running%20Apache%20Spark%20On%20Kubernetes.ipynb)

The instructions are basically the same as [Create A SparkContext For Locally Hosted Cluster](Create%20A%20SparkContext%20For%20Locally%20Hosted%20Cluster.ipynb)

## Adjenda
1. Create SparkContext
2. Cleanup Spark and Kubernetes
3. Package This As A Helper Module



# 1. Create SparKContext Object

We can see that there are no pods running on our kubernetes cluster

In [2]:
!kubectl -n spark get pod

We create the SparkContext

In [3]:
import pyspark

In [4]:
# Set some vars to specify where the kubernetes master is
kubernetes_master_ip = "15.4.7.11"
kubernetes_master_port = "6443"
spark_master_url = "k8s://https://{0}:{1}".format(kubernetes_master_ip, kubernetes_master_port)

In [5]:
# Determine the ip address of the machine
import netifaces
import re
nic_uuid = netifaces.gateways()['default'][netifaces.AF_INET][1]
nic_details = netifaces.ifaddresses(nic_uuid)
ip_address = None
for i, nic_detail, in nic_details.items():
    if all([key in nic_detail[0].keys() for key in ["addr", "netmask", "broadcast"]]):
        if re.match("([0-9]+\\.)+", nic_detail[0]["addr"]):
            ip_address = nic_detail[0]["addr"]
            break
print("The ip was detected as: {0}".format(ip_address))

The ip was detected as: 15.4.12.12


In [7]:
# Wire up the SparkConf object
sparkConf = pyspark.SparkConf()
sparkConf.setMaster(spark_master_url)

sparkConf.setAppName("spark-jupyter-win")

sparkConf.set("spark.submit.deploy.mode", "cluster")
sparkConf.set("spark.kubernetes.container.image", "tschneider/apache-spark-k8:v7") 
sparkConf.set("spark.kubernetes.namespace", "spark")
sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-sa")
sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark-sa")

sparkConf.set("spark.executor.instances", "3")
sparkConf.set("spark.executor.cores", "2")
sparkConf.set("spark.executor.memory", "1024m")
sparkConf.set("spark.driver.memory", "1024m")

# If we are not using a hostname registered with a dns server, we need to set this parameter
sparkConf.set("spark.driver.host", ip_address)

In [8]:
sparkConf.getAll()

dict_items([('spark.master', 'k8s://https://15.4.7.11:6443'), ('spark.app.name', 'spark-jupyter-win'), ('spark.submit.deploy.mode', 'cluster'), ('spark.kubernetes.container.image', 'tschneider/apache-spark-k8:v7'), ('spark.kubernetes.namespace', 'spark'), ('spark.kubernetes.pyspark.pythonVersion', '3'), ('spark.kubernetes.authenticate.driver.serviceAccountName', 'spark-sa'), ('spark.kubernetes.authenticate.serviceAccountName', 'spark-sa'), ('spark.executor.instances', '3'), ('spark.executor.cores', '2'), ('spark.executor.memory', '1024m'), ('spark.driver.memory', '1024m'), ('spark.driver.host', '15.4.12.12')])

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
sc = spark.sparkContext

22/01/15 17:18:27 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 15.4.12.12 instead (on interface eth0)
22/01/15 17:18:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/15 17:18:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


We can look at kubernetes to see that out worker nodes were created.

In [12]:
! kubectl -n spark get pod

NAME                                        READY     STATUS    RESTARTS   AGE
spark-jupyter-win-f6bab17e5ebf7511-exec-1   1/1       Running   0          26m
spark-jupyter-win-f6bab17e5ebf7511-exec-2   1/1       Running   0          26m
spark-jupyter-win-f6bab17e5ebf7511-exec-3   1/1       Running   0          26m


# 2. Cleanup Spark Cluster On Kubernetes
When done working with spark we need to cleanup we kubernetes objects that were dynamically created.

In [13]:
sc.stop()

In [14]:
! kubectl -n spark get pod

NAME                                        READY     STATUS        RESTARTS   AGE
spark-jupyter-win-f6bab17e5ebf7511-exec-1   1/1       Terminating   0          26m
spark-jupyter-win-f6bab17e5ebf7511-exec-2   1/1       Terminating   0          26m
spark-jupyter-win-f6bab17e5ebf7511-exec-3   1/1       Terminating   0          26m


# 3. Package This As A Helper Module

I have packaged the code above into a helper module. We can include this module as a way to have this code execute in a neat and standard way. Simply execute the following in a cell:

In [18]:
# Load a helper module
import importlib.util
spec = importlib.util.spec_from_file_location("spark_helper", "../../../Utilities/spark_helper.py")
spark_helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(spark_helper)

In [21]:
spark_app_name = "spark-jupyter-win"
docker_image = "tschneider/pyspark:v5"
k8_master_ip = "15.4.7.11"
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

Setting SPARK_HOME
/usr/lib/spark-3.1.1-bin-hadoop2.7

Running findspark.init() function
['/usr/lib/spark-3.1.1-bin-hadoop2.7/python', '/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip', '/ml-training-jupyter-notebooks/Machine Learning/Big Data And Big Compute/Apache Spark', '/tmp/spark-b3403cd8-7c96-44ec-bcdc-e435cd31216e/userFiles-4d235b3e-7195-4ca3-8159-d24ec4ddbd04', '/usr/local/lib/python39.zip', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/site-packages']

Setting PYSPARK_PYTHON
/usr/local/bin/python3

Configuring URL for kubernetes master
k8s://https://15.4.7.11:6443

Determining IP Of Server
The ip was detected as: 15.4.12.12

Creating SparkConf Object
('spark.executor.instances', '3')
('spark.kubernetes.container.image', 'tschneider/pyspark:v5')
('spark.app.name', 'spark-jupyter-win')
('spark.driver.host', '15.4.12.12')
('spark.executor.memoryOverhead', '1024m')
('spark.driver.memory', '1024m')
('spark.execu

When done working with spark we need to cleanup we kubernetes objects that were dynamically created.

In [ ]:
sc.stop()